In [1]:
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, time, datetime

import os
from glob import glob
from scipy.integrate import trapz
import matplotlib.cm as cm
import locale
from dateutil.parser import *
from matplotlib.ticker import FuncFormatter, MaxNLocator
from functools import partial
import matplotlib as mpl

from dateutil.parser import *

from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)

import matplotlib.dates as mdates

pd.plotting.register_matplotlib_converters()

d:\profile\mlb\eigene dateien\programming\programming\uvvisanalysis\venv\lib\site-packages\ipykernel_launcher.py:19: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.


In [2]:
potentiostatData = pd.read_csv('Potentiostate/MergedFile.csv')
potentiostatData['actualTime'] = potentiostatData['actualTime'].map(lambda x: parse(x).timestamp())

# figure1 = plt.figure()
# ax1 = figure1.add_subplot(1, 1, 1)
# ax2 = ax1.twinx()

# ax1.plot(potentiostatData['actualTime'], potentiostatData['Im'], c='r')
# ax2.plot(potentiostatData['actualTime'], potentiostatData['Vf'], c='b')

In [3]:
def lookupPotentialFromDate(lookupDatafile, lookupTime):
    return lookupDatafile['Vf'][lookupDatafile['actualTime'] == lookupTime].mean()

def lookupCurrentFromDate(lookupDatafile, lookupTime):
    return lookupDatafile['Im'][lookupDatafile['actualTime'] == lookupTime].mean()

In [4]:
I0 = pd.read_csv('Baseline References/WUV.txt', delim_whitespace=True, header=None, skiprows=14, names=['wavelength', 'intensity'], dtype={'wavelength':  np.float64, 'intensity': np.float64}, decimal=",")
I0['intensity_norm'] = I0['intensity'] / I0[I0['wavelength'] > 500]['intensity'].max()
# I0.plot('wavelength', 'intensity_norm')

I02 = pd.read_csv('Baseline References/WUV.txt', delim_whitespace=True, header=None, skiprows=14, names=['wavelength', 'intensity'], dtype={'wavelength':  np.float64, 'intensity': np.float64}, decimal=",")
I02['intensity_norm'] = I02['intensity'] / I02[I02['wavelength'] > 500]['intensity'].max()
I02['absorption'] = I02['intensity_norm'] / I0['intensity_norm']
# I02.plot('wavelength', 'intensity_norm')
# I02.plot('wavelength', 'absorption')

LEDSignal = pd.read_csv('Baseline References/WhiteLED.txt', delim_whitespace=True, header=None, skiprows=14, names=['wavelength', 'intensity'], dtype={'wavelength':  np.float64, 'intensity': np.float64}, decimal=",")
LEDSignal['intensity_norm'] = LEDSignal['intensity'] / LEDSignal[LEDSignal['wavelength'] > 200]['intensity'].max()
LEDSignal.plot('wavelength', 'intensity_norm')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='wavelength'>

In [5]:
BroadLEDSignal = pd.read_excel('Baseline References/MBB1F1_data.xlsx', sheet_name='MBB1F1 Spectrum', header=1, usecols='C,D', names=['wavelength', 'intensity'], na_filter=None)
BroadLEDSignal.plot('wavelength', 'intensity')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='wavelength'>

In [6]:
XenonLampSignal = pd.read_excel('Baseline References/SLS402_Data.xlsx', sheet_name='SLS402 Spectrum', header=2, usecols='C, D', names=['wavelength', 'intensity'])
XenonLampSignal['intensity_norm'] = XenonLampSignal['intensity'] * 4.5
XenonLampSignal.plot('wavelength', 'intensity_norm', ylim=(0, 1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='wavelength'>

In [7]:
PATH = 'UVVis'
EXT = '*.txt'
filterString = ''
allSpectrometerFilenames = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in sorted(glob(os.path.join(path, EXT)), key=os.path.getmtime)]

spectrometerFiles = {}

selectedFIles = (0, 884)

for filename in allSpectrometerFilenames[selectedFIles[0]:selectedFIles[1]]:
    extractedDate = None
    with open(filename, mode='rt') as file:
        lines = file.readlines()
        dateline = lines[2].strip()[6:]
        extractedDate = parse(dateline)

    data = pd.read_csv(filename, delim_whitespace=True, header=None, skiprows=14, names=['wavelength', 'intensity'], dtype={'wavelength':  np.float64, 'intensity': np.float64}, decimal=",")
    extractedDate = extractedDate.timestamp()
    data['time'] = extractedDate

    spectrometerFiles[extractedDate] = data

d:\profile\mlb\eigene dateien\programming\programming\uvvisanalysis\venv\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [8]:
startTime = list(spectrometerFiles.items())[0][1]['time']
startTime

0       1.608125e+09
1       1.608125e+09
2       1.608125e+09
3       1.608125e+09
4       1.608125e+09
            ...     
3643    1.608125e+09
3644    1.608125e+09
3645    1.608125e+09
3646    1.608125e+09
3647    1.608125e+09
Name: time, Length: 3648, dtype: float64

In [9]:
for measurementdate, measurement in spectrometerFiles.items():
    measurement['intensity_smoothed'] = measurement['intensity'].rolling(200, win_type='nuttall').mean().shift(-100).fillna(1)
    integrateValue = trapz(measurement['intensity'], x=measurement['wavelength'])
    measurement['intensity_smoothed_normArea'] = measurement['intensity_smoothed'] / integrateValue
    measurement['intensity_smoothed_normMax'] = measurement['intensity_smoothed'] / measurement['intensity_smoothed'].max()
    measurement['intensity_normArea'] = measurement['intensity'] / integrateValue
    measurement['intensity_normMax'] = measurement['intensity'] / measurement['intensity_smoothed'].max()
    
    
    
    measurement['potential'] = lookupPotentialFromDate(potentiostatData, measurementdate)
    measurement['potential'] = measurement['potential'].fillna(0)
    
    measurement['current'] = lookupCurrentFromDate(potentiostatData, measurementdate)
    measurement['current'] = measurement['current'].fillna(0)
    
    measurement['deltaTime'] = measurement['time'] - startTime
    
    measurement['absorption'] = -np.log(measurement['intensity_smoothed_normMax'] / I0['intensity_norm'])
    measurement['absorption_smoothed'] = measurement['absorption'].rolling(50, win_type='nuttall').mean().shift(-25).fillna(1)
    
    
    measurement['intensity_LEDconv'] = measurement['intensity_smoothed_normMax'] * LEDSignal['intensity_norm']
    measurement['intensity_broadLEDconv'] = measurement['intensity_smoothed_normMax'] * BroadLEDSignal['intensity']
    measurement['intensity_Xenonconv'] = measurement['intensity_smoothed_normMax'] * XenonLampSignal['intensity_norm']
    

d:\profile\mlb\eigene dateien\programming\programming\uvvisanalysis\venv\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\profile\mlb\eigene dateien\programming\programming\uvvisanalysis\venv\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\profile\mlb\eigene dateien\programming\programming\uvvisanalysis\venv\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\profile\mlb\eigene dateien\programming\programming\uvvisanalysis\venv\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\profile\mlb\eigene dateien\programming\programming\uvvisanalysis\venv\lib\site-packages\pandas\core\array

In [10]:
figure1 = plt.figure()
ax1 = figure1.add_subplot(1, 1, 1, projection='3d')

for measurementdate, measurement in list(spectrometerFiles.items())[0:450]:
    ax1.plot(measurement['wavelength'], measurement['absorption'], measurementdate, zdir='y')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
yTimeLabels = [measurement['deltaTime'].min()/60 for key, measurement in list(spectrometerFiles.items())]

In [12]:
X = pd.Series(list(spectrometerFiles.items())[0][1]['wavelength'])
Y = pd.Series(list(spectrometerFiles.keys()))
XX, YY = np.meshgrid(X, Y)

Z_intensity = np.array([spectrometerFiles[yGrid[0]]['intensity'] for yGrid in YY])
Z_intensity_smoothed = np.array([spectrometerFiles[yGrid[0]]['intensity_smoothed_normMax'] for yGrid in YY])
Z_current = np.array([spectrometerFiles[yGrid[0]]['current'] for yGrid in YY])
Z_potential = np.array([spectrometerFiles[yGrid[0]]['potential'] for yGrid in YY])
Z_absorption = np.array([spectrometerFiles[yGrid[0]]['absorption_smoothed'] for yGrid in YY])
Z_LEDconv = np.array([spectrometerFiles[yGrid[0]]['intensity_LEDconv'] for yGrid in YY])

Y = pd.Series(yTimeLabels)
XX, YY = np.meshgrid(X, Y)

In [13]:
figure3 = plt.figure()

ax5 = figure3.add_subplot(1,6,1)
ax5.set_xlim(200, 860)
cont = ax5.pcolormesh(XX, YY, Z_intensity, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto')

ax5 = figure3.add_subplot(1,6,2)
# ax5.set_xlim(420, 864)
ax5.get_yaxis().set_visible(False)
cont = ax5.pcolormesh(XX, YY, Z_intensity_smoothed, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto')

ax6 = figure3.add_subplot(1,6,3)
ax6.set_xlim(200, 860)
ax6.get_yaxis().set_visible(False)
cont = ax6.pcolormesh(XX, YY, Z_potential, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto')

ax7 = figure3.add_subplot(1,6,4)
ax7.set_xlim(200, 860)
ax7.get_yaxis().set_visible(False)
cont = ax7.pcolormesh(XX, YY, Z_current, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto')

ax8 = figure3.add_subplot(1,6,5)
ax8.set_xlim(200, 860)
ax8.get_yaxis().set_visible(False)
cont = ax8.pcolormesh(XX, YY, Z_absorption, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto', vmin=-2.5, vmax=2)

ax9 = figure3.add_subplot(1,6,6)
ax9.set_xlim(200, 860)
ax9.get_yaxis().set_visible(False)
cont = ax9.pcolormesh(XX, YY, Z_LEDconv, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

d:\profile\mlb\eigene dateien\programming\programming\uvvisanalysis\venv\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  
d:\profile\mlb\eigene dateien\programming\programming\uvvisanalysis\venv\lib\site-packages\ipykernel_launcher.py:12: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  if sys.path[0] == '':
d:\profile\mlb\eigene dateien\programming\programming\uvvisanalysis\venv\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or d

In [14]:
X = pd.Series(list(spectrometerFiles.items())[0][1]['wavelength'])
Y = pd.Series(list(spectrometerFiles.keys()))
XX, YY = np.meshgrid(X, Y)

Z_absorption = np.array([spectrometerFiles[yGrid[0]]['absorption_smoothed'] for yGrid in YY])

ax8.clear()
# ax8.set_xlim(410, 864)
cont = ax8.pcolormesh(XX, YY, Z_absorption, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto', vmin=-2, vmax=5)
Y = pd.Series(yTimeLabels)
XX, YY = np.meshgrid(X, Y)


d:\profile\mlb\eigene dateien\programming\programming\uvvisanalysis\venv\lib\site-packages\ipykernel_launcher.py:10: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  # Remove the CWD from sys.path while we load stuff.


In [15]:
controlDataDate = list(spectrometerFiles.keys())
controlDataPotential = [file['potential'].mean() for key, file in spectrometerFiles.items()]
controlDataCurrent = [file['current'].mean()/1E-6 for key, file in spectrometerFiles.items()]

figure5 = plt.figure()
ax8 = figure5.add_subplot(1, 1, 1)
ax9 = ax8.twinx()

ax8.plot(controlDataDate, controlDataPotential, c='b')
ax9.plot(controlDataDate, controlDataCurrent, c='r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
figure6 = plt.figure()
ax10 = figure6.add_subplot(1, 2, 1)
ax11 = figure6.add_subplot(1, 2, 2)
ax12 = ax11.twinx()

ax10.set_xlim(200, 900)
ax10.set_ylim(0, 1.5)
ax11.set_xlim(200, 900)
ax11.set_ylim(0, 1.5)
ax12.set_xlim(200, 900)
ax12.set_ylim(0, 1.5)


spectra = list(spectrometerFiles.items())

# ax10.plot(XenonLampSignal['wavelength'], XenonLampSignal['intensity_norm'])
# ax12.plot(XenonLampSignal['wavelength'], XenonLampSignal['intensity_norm'])

for measurementdate, measurement in [spectra[500], spectra[540], spectra[450]]:
    ax10.plot(measurement['wavelength'], measurement['intensity_smoothed_normMax'])
    ax11.plot(measurement['wavelength'], measurement['intensity_Xenonconv'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
spectraAverage = spectra[520:521]
totalIntensity = spectraAverage[0][1]['intensity'] * 0

for ispectra in spectraAverage:
    totalIntensity = totalIntensity + ispectra[1]['intensity']
    
totalIntensity = totalIntensity / len(spectraAverage)
totalIntensity = totalIntensity.rolling(50, win_type='nuttall').mean().shift(-25).fillna(1)

figure7 = plt.figure()
ax13 = figure7.add_subplot(1, 1, 1)

ax13.plot(spectraAverage[0][1]['wavelength'], totalIntensity)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
spectraAverage = spectra[520:550]
totalIntensity = spectraAverage[0][1]['intensity'] * 0

for ispectra in spectraAverage:
    totalIntensity = totalIntensity + ispectra[1]['intensity']
    
totalIntensity = totalIntensity / len(spectraAverage)
totalIntensity = totalIntensity.rolling(50, win_type='nuttall').mean().shift(-25).fillna(1)

ax13.plot(spectraAverage[0][1]['wavelength'], totalIntensity)